## Simulating beam-coupling impedance and wakefields
### The CERN-LHC bellows example

In this demonstration notebook, the theory behing the numerical calculation of electromagnetic wakefields will be introduced with the example of a **real** element: the LHC bellows.

The bellows are vacuum components installed between rigid beampipe sections to accomodate thermal expansion and decouple mechanical stresses. They are thin-wall corrugated elements, presenting convolutions that grant them flexibility.

Due to the convolutions, they are electromagnetically non-smooth, having therefore a considerable impact on beam-coupling impedance.


<div style="text-align:center">
  <img src="data/lhc-bellows.png" width="400">
</div>


### Tools to use:

There are several codes to compute beam-coupling impedance: in time-domain or frequency domain, numerical or semi-analytical, co-moving window or resting frame... To perform full-wave 3D simulations on arbitrary geometry and materials, the reference code is the commercial CST Studio Wakefield Solver.

In this demonstration we will use [Wakis](https://github.com/ImpedanCEI/wakis), a new open-source 3D electromagnetic code built in Python, that can perform full-wave 3D simulations solving the general form of Maxwell's Equations in time-domain:

$$
\begin{align}
\oint_{\partial A} \mathbf{E}\cdot \mathrm{d}\mathbf{s} &= -\iint_{A}\frac{\partial \mathbf{B}}{\partial t}\cdot \mathrm{d}\mathbf{A} \tag{1a}\\[6pt]
\oint_{\partial A} \mathbf{H}\cdot \mathrm{d}\mathbf{s} &= \iint_{A}\left(\frac{\partial \mathbf{D}}{\partial t} + \mathbf{J}\right)\cdot \mathrm{d}\mathbf{A} \tag{1b}\\[6pt]
\oiint_{\partial V} \mathbf{B}\cdot \mathrm{d}\mathbf{A} &= 0 \tag{1c}\\[6pt]
\oiint_{\partial V} \mathbf{D}\cdot \mathrm{d}\mathbf{A} &= \iiint_{V}\rho\, \mathrm{d}V \tag{1d}\\[6pt]
\mathbf{D} = \varepsilon \mathbf{E},\quad 
\mathbf{B} &= \mu \mathbf{H},\quad 
\mathbf{J} = \sigma \mathbf{E} + \rho\mathbf{v} \tag{1e}
\end{align}
$$

With a passing beam current modeled as a Gaussian distribution:

$$
\mathbf{J}_z(x_{\text{src}}, y_{\text{src}}, \vec{z}, t) \equiv \lambda(s) =
\frac{q \beta c}{\sqrt{2\pi} \sigma_z} \, 
\exp\left( -\frac{(\vec{s} - s_0)^2}{2\sigma_z^2} \right)
$$

with:
- $\vec{s} = \vec{z} - \beta c t$: beam-frame coordinate
- $s_0 = z_{\min} - \beta c t_{\text{inj}}$: center of bunch
- $q$ the charge in $\text{nC}$
- $\sigma_z$ the bunch length in $\text{m}$

By computing the fields in time-domain, we can integrate them to obtain the wakefields and impedance:

$$
W(s) = \frac{1}{q_s} \int_{-\infty}^{\infty} \left[ E_z(z, t) + \beta c \, \vec{e}_z \times \vec{B}(z, t) \right]_{t = (s + z)/c} \, dz
$$


The wakefield is the input to beam-dynamics simulations. Its Fourier transform yields the **beam-coupling impedance**, in frequency domain:

$$
Z(\omega) = \beta c \cdot \frac{\mathcal{F}[W(s)]}{\mathcal{F}[\lambda(s)]}
$$

----

In [ ]:
# Import required libraries for geometry, visualization, and simulation
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt

from wakis import SolverFIT3D
from wakis import GridFIT3D
from wakis import WakeSolver

pv.global_theme.window_size = [800, 400]
plot_pyvista = False

In [ ]:
# ---------- Domain & Mesh setup ---------
# Set up geometry & materials dictionaries
stl_file = 'data/bellow-vacuum-mm.stl'

# Domain bounds
bellow = pv.read(stl_file).scale(1e3)
xmin, xmax, ymin, ymax, zmin, zmax = bellow.bounds

# Number of mesh cells
Nx = 60
Ny = 60
Nz = 200
print(f'Total number of mesh cells: {Nx*Ny*Nz}')

# set grid and geometry
grid = GridFIT3D(*bellow.bounds, 
                Nx, Ny, Nz,
                stl_solids={'bellow': stl_file},
                stl_materials={'bellow': 'vacuum'},
                stl_scale = 1e3,
                )
# visualize grid
if plot_pyvista:
    grid.inspect()

Total number of mesh cells: 720000
Generating grid with 720000 mesh cells...
Importing STL solids...
Total grid initialization time: 8.031729698181152 s


Widget(value='<iframe src="http://localhost:39357/index.html?ui=P_0x754c47e9e9d0_3&reconnect=auto" class="pyvi…

### [EXTRA] Parametric scans:

One can generate the geometry direcly from Python to perform parametric scans on the different bellow construction paramters:

In [ ]:
# --- Generate beam pipe geometry ---
r_pipe = 24e-3      # Pipe radius in metersm
l_pipe = 320e-3     # Pipe length in metersth in meters
pipe = pv.Cylinder(center=(0,0,0), direction=(0,0,1), radius=r_pipe, height=l_pipe)

# --- Generate bellow convolutions ---
r_conv = 30e-3   # convolution radius [m]
l_conv = 4e-3    # length of each convolution [m]
n_convs = [25, 18, 15]      # number of convolutions
l_between_conv = 4e-3  # length between convolutions [m]

# Parametric generation of geometry
bellows = []
for n_conv in n_convs:
    print(f'Generating bellow with {n_conv} convolutions...')
    convolutions = []
    z_start = n_conv//2*(l_conv + l_between_conv) - l_conv # start of the convolutions [m]
    for n in range(n_conv):
        z_start_n_conv = -z_start+n*(l_conv+l_between_conv)
        conv = pv.Cylinder(center=(0,0,z_start_n_conv), # center of the convolution
                           direction=(0,0,1),           # z-direction
                           radius=r_conv,
                           height=l_conv)
        convolutions.append(conv)                       # append to list

    # Sum the generated geometry
    pipe = pipe.triangulate()                          # triangulate pipe
    convolutions = np.sum(convolutions).triangulate()  # triangulate convolutions
    bellow = pipe | convolutions  # union of meshes without internal faces
    bellow.save(f'data/{n_conv}_convolution_bellow.stl')  # save intermediate geometry
    bellows.append(bellow)

if plot_pyvista:
    pl = pv.Plotter()   
    pl.add_mesh(bellows[0], color='white', edge_color='black', label=f'{n_convs[0]} convolutions') 
    pl.add_mesh(bellows[1].translate((0,-0.1,0)), color='white', edge_color='black', label=f'{n_convs[1]} convolutions')
    pl.add_mesh(bellows[2].translate((0,0.1,0)), color='white', edge_color='black', label=f'{n_convs[2]} convolutions')
    pl.show()